<a href="https://colab.research.google.com/github/mdjamina/model_relatives_psr_fr/blob/text_class_model/psr_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation modules

In [ ]:
!pip install datasets
!pip install -U transformers[torch] accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_6

# Chargement du dataset

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

#chargement de dataset à partir de Hugging Face
raw_datasets = load_dataset("djamina/relatives_psr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/99 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/55 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'texts', 'tokens', 'labels', 'psr_tags', 'psr_seq_tags'],
        num_rows: 392
    })
    validation: Dataset({
        features: ['id', 'texts', 'tokens', 'labels', 'psr_tags', 'psr_seq_tags'],
        num_rows: 99
    })
    test: Dataset({
        features: ['id', 'texts', 'tokens', 'labels', 'psr_tags', 'psr_seq_tags'],
        num_rows: 55
    })
})

# Préparation des données

In [ ]:
task = "labels" # Should be one of "psr" or "psr_seq"

model_checkpoint = "almanach/camembert-base"
batch_size = 16

## Tokenizer




In [ ]:
 #Importation de la classe AutoTokenizer de la bibliothèque 'transformers'
from transformers import CamembertTokenizerFast


tokenizer = CamembertTokenizerFast.from_pretrained(model_checkpoint, force_download=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

In [ ]:
# Vérification si le tokenizer est une version rapide

import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

### Traitement des labels

In [ ]:
# Explication de la sortie des tokens :
# - '<s>' : Token de début de séquence (utilisé pour indiquer le début d'une phrase).
# - Les caractères '▁' indiquent le début d'un nouveau mot ou d'un sous-mot, ceci est dû à l'utilisation de la tokenisation par sous-mots (subword tokenization) utilisée par le modèle CamemBERT.
# - Les sous-mots permettent de gérer efficacement les mots rares ou inconnus en les décomposant en unités plus petites.
# - Par exemple, '▁Mes' est le mot "Mes", '▁collègues' est le mot "collègues", mais 's', 'cell', 'ons' sont des sous-mots formant le mot "scellons".
# - Cette méthode améliore la capacité du modèle à généraliser à partir de morceaux de mots, surtout en présence de vocabulaire diversifié.


In [ ]:

# Extraction des features 'psr_tags' à partir du dataset
label_names = raw_datasets["train"].features[f"{task}"]
print(f'label_names : {label_names}')
print("-----------------")

words = raw_datasets["train"][15]["tokens"]
labels = raw_datasets["train"][15][f"{task}"]

print(f'words : {words}')
print("-----------------")

print(f'labels : {labels}')



label_names : ClassLabel(names=['O', 'DET', 'APPO', 'AMBIGUE'], id=None)
-----------------
words : ['Notre', 'peuple', "n'", 'est', 'pas', 'formé', "d'", 'un', 'peu', 'plus', 'de', 'soixante', 'millions', "d'", 'individus', 'qui', 'cohabiteraient', '.', 'Il', 'est', 'indivisible', ',', 'précisément', 'parce', 'que', 'ce', 'qui', 'le', 'tient', 'est', 'plus', 'fort', 'que', 'de', 'simples', 'règles', 'ou', 'des', 'organisations', '.', "C'", 'est', 'un', 'engagement', 'chaque', 'jour', 'répété', 'qui', 'fait', 'que', 'notre', 'citoyenneté', "n'", 'est', 'jamais', 'abstraite', 'et', 'froide', ',', 'mais', 'qu', "'", 'elle', 'est', 'pleine', 'et', 'entière', 'par', 'ce', 'lien', 'fraternel', 'qui', 'nous', 'unit', 'et', 'dont', 'nous', 'devons', 'retrouver', 'la', 'vigueur', '.']
-----------------
labels : 1


In [ ]:
# cette méthode permet de tokeniser le dataset (par ligne) et d'aligner les labels avec les tokens générés par le tokenizer

def tokenize_and_align_labels(rows):
    tokenized_inputs = tokenizer(
        rows["tokens"], truncation=True, is_split_into_words=True
    )

    return tokenized_inputs




### Tokenization

Application de la méthode 'tokenize_and_align_labels' sur le dataset 'raw_datasets'


pour obtenir le dataset 'tokenized_datasets' qui sera utilisé pour l'entraînement du modèle

In [ ]:
# psr_tags

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=[c for c in raw_datasets["train"].column_names if c not in ["labels"]],
)

Map:   0%|          | 0/392 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 392
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 99
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 55
    })
})

# Evaluate

In [ ]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


# Train

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(3)])
batch

{'labels': tensor([2, 1, 2]), 'input_ids': tensor([[    5,    50,  1110,    73,    36,  2616,   938,    18,    11,  8148,
         12195,    15,   789,    20,  1964, 17305,   174,    21,     7, 23122,
            21,     7,    51,   111, 13322,   287,    21,     9,     6,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1],
        [    5,  1444,  3847,    56,    22,   340,  9398,    21,     9,    61,
          4011,  1547,    33,  1897,    14,  1866,   143,   405,    21,    11,
            21,   265,    21,     9,   799,   320,    31, 20987,    13,  5581,
            24,    22,    21, 20105,    28,  5018,    35,

In [ ]:
raw_datasets["train"].features

{'id': Value(dtype='int32', id=None),
 'texts': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': ClassLabel(names=['O', 'DET', 'APPO', 'AMBIGUE'], id=None),
 'psr_tags': Sequence(feature=ClassLabel(names=['O', 'DET', 'APPO', 'AMBIGUE'], id=None), length=-1, id=None),
 'psr_seq_tags': Sequence(feature=ClassLabel(names=['O', 'B-DET', 'I-DET', 'B-APPO', 'I-APPO', 'B-AMBIGUE', 'I-AMBIGUE'], id=None), length=-1, id=None)}

In [ ]:

label_names = raw_datasets["train"].features[f"{task}"].names

id2label = {x:y for x,y in enumerate(label_names)}
label2id = {y:x for x,y in enumerate(label_names)}
label2id

{'O': 0, 'DET': 1, 'APPO': 2, 'AMBIGUE': 3}

In [ ]:
from transformers import CamembertConfig, AutoModelForSequenceClassification

# Récupérer les noms des labels

num_labels = len(label_names)

# Définir la configuration du modèle
config = CamembertConfig(
    vocab_size=500_000,  # Taille du vocabulaire
    num_labels=num_labels,  # Nombre de labels dans votre dataset
    hidden_size=768,  # Taille des embeddings
    num_attention_heads=12,  #le lien entre les tokens, jusqu'au 12 eme token
    num_hidden_layers=12,  # Nombre de couches
)

config.label2id = label2id
config.id2label = id2label

# Initialiser le modèle avec la configuration
model = AutoModelForSequenceClassification.from_config(config)

In [ ]:
model.config.num_labels

4

In [ ]:
label_names

['O', 'DET', 'APPO', 'AMBIGUE']

In [ ]:
from transformers import TrainingArguments

model_name =  f"relatives_{task}"



args = TrainingArguments(
    model_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    push_to_hub=False,

)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [ ]:
history = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.839290,0.414141,0.000000
2,No log,0.756993,0.585859,0.738854
3,No log,0.686720,0.585859,0.738854
4,No log,0.683456,0.585859,0.738854
5,No log,0.683998,0.575758,0.704225


<ipython-input-22-b1430cee0b64>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

events.out.tfevents.1717789126.e2a7f679267f.866.1:   0%|          | 0.00/7.08k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/djamina/relatives_labels/commit/8b5659f7b53d754ebecdff5619773973ea95de6d', commit_message='Training complete', commit_description='', oid='8b5659f7b53d754ebecdff5619773973ea95de6d', pr_url=None, pr_revision=None, pr_num=None)

## Inference